这一教程我们来探索利用 xalpha 提供的工具来计算可转债定价

In [5]:
import xalpha as xa
import pandas as pd
import numpy as np

xa.set_display("notebook")

<IPython.core.display.Javascript object>

我们的对比基准是富投网提供的可转债数据, 其具有详细的可转债定价数据

In [2]:
df = xa.misc.get_ri_status()
df

,序号,转债名称,转债代码,进展/上市日期,股票名称,股票代码,行业,子行业,转债价格,涨跌,...,历史波动,隐含波动,预测波动,弹性,信用,折现率,双低指数,MA20乖离,热门度,转债名称
0,1,春秋转债,113577,2020-05-13,春秋电子,603890,电子,电子行业,119.14,-0.68%,...,61%,14%,18%,0.70,AA-,5.33%,131,-1.9%,3,春秋转债
1,2,N聚飞转,123050,2020-05-15,聚飞光电,300303,电子,LED行业,121.7,21.7%,...,56%,<2%,17%,0.73,AA-,5.33%,128,13.0%,6,N聚飞转
2,3,维尔转债,123049,2020-05-12,维尔利,300190,城市公用设施,三废处理,112,-0.53%,...,52%,17%,17%,0.50,AA-,5.33%,133,-1.8%,8,维尔转债
3,4,凌钢转债!,110070,2020-05-13,凌钢股份,600231,钢铁,炼钢,100.81,-0.98%,...,40%,16%,18%,0.35,AA,4.33%,124,-1.9%,6,凌钢转债!
4,5,华统转债,128106,2020-05-08,华统股份,002840,农业,农产品加工,121.6,-1.22%,...,48%,20%,16%,0.72,AA,4.33%,135,3.5%,5,华统转债
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,245,广汽转债!,113009,2016-02-04,广汽集团,601238,汽车,乘用车,112,-0.48%,...,37%,47%,15%,0.02,AAA,2.38%,155,0.1%,14,广汽转债!
245,246,九州转债!,110034,2016-01-29,九州通,600998,医药,医药商业,117.17,-1.26%,...,28%,28%,15%,0.36,AA+,2.88%,137,-4.0%,29,九州转债!
246,247,国贸转债!,110033,2016-01-19,厦门国贸,600755,交通运输,交通运输行业,110.06,-0.04%,...,33%,27%,15%,0.12,AAA,2.38%,136,-0.4%,18,国贸转债!
247,248,航信转债!!,110031,2015-06-30,航天信息,600271,计算机及应用,软件开发,111.1,-0.14%,...,40%,40%,17%,0.04,AAA,2.38%,144,-6.9%,15,航信转债!!


xalpha 提供的转债价格计算器债券价值部分估算是非常好的，期权部分只采用了简单的 BS 公式估算 call 价值，其他部分都省略，让我们看下这样的前提下的估值效果和富投网以及市场情况差多少。

In [9]:
# 首先我们看下 xalpha 的可转债计算器怎么用
c = xa.CBCalculator("SH113575")
c.analyse()

{'bond_value': 92.10326407496153,
 'bondrate': 0.036832,
 'cbcode': 'SH113575',
 'cbprice': 112.56,
 'date': '2020-05-17',
 'enddate': '2026-04-08',
 'historical_valatility': 0.3943500675205567,
 'interest': [0.4, 0.6, 1.0, 1.5, 2.0, 2.3, 105.7],
 'issuedate': '2020-04-08',
 'name': '东时转债',
 'option_value': 24.42487275792871,
 'predicted_volatility': 0.19,
 'premium': -3.405303595114395,
 'rating': 'AA',
 'riskfreerate': 0.019702,
 'stockcode': 'SH603377',
 'stockprice': 15.01,
 'tot_value': 116.52813683289024,
 'years': 5.8931506849315065,
 'ytm_wi_tax': -0.0009515432823119259,
 'ytm_wo_tax': 0.0014396299788003476,
 'zgj': 14.76}

没错用起来就是这么简单，所有的无风险利率，波动率，不同信用等级和久期对应的债券利率等细节都自动处理好了。
你还可以了解历史任意一天的可转债价值情况。不过值得提醒的是，暂时引擎未处理可转债转股价调整信息，总是利用现在的转股价估值，可能会带入未来数据。

In [10]:
c.analyse("2020-05-01")

{'bond_value': 93.18697238786118,
 'bondrate': 0.034732,
 'cbcode': 'SH113575',
 'cbprice': 117.45,
 'date': '2020-05-01',
 'enddate': '2026-04-08',
 'historical_valatility': 0.3928622592240376,
 'interest': [0.4, 0.6, 1.0, 1.5, 2.0, 2.3, 105.7],
 'issuedate': '2020-04-08',
 'name': '东时转债',
 'option_value': 27.92990803084096,
 'predicted_volatility': 0.19,
 'premium': -3.0275552061989175,
 'rating': 'AA',
 'riskfreerate': 0.017886,
 'stockcode': 'SH603377',
 'stockprice': 15.82,
 'tot_value': 121.11688041870214,
 'years': 5.93972602739726,
 'ytm_wi_tax': -0.008246198442578546,
 'ytm_wo_tax': -0.005899073653703114,
 'zgj': 14.76}

In [3]:
df["bvalue"] = 0
df["ovalue"] = 0
df["tvalue"] = 0
df["votality"] = 0
df["rate"] = 0
for i, r in df.iterrows():
    code = xa.universal.ttjjcode(r["转债代码"])
    c = xa.CBCalculator(code)
    d = c.analyse()
    df.loc[i, "bvalue"] = d["bond_value"]
    df.loc[i, "ovalue"] = d["option_value"]
    df.loc[i, "tvalue"] = d["tot_value"]
    df.loc[i, "votality"] = d["predicted_volatility"]
    df.loc[i, "rate"] = d["bondrate"]

the format of rlist must contain the final return more than 100 without interest of that year
the format of rlist must contain the final return more than 100 without interest of that year
the format of rlist must contain the final return more than 100 without interest of that year


In [10]:
df["ridiff"] = pd.to_numeric(df["内在价值"]) - pd.to_numeric(df["转债价格"])
np.std(sorted(df["ridiff"])[10:-10]), np.mean(sorted(df["ridiff"])[10:-10])
# 富投估值和市价比较

(6.981658410811314, -2.9449737991266383)

In [11]:
df["ridiff"] = pd.to_numeric(df["tvalue"]) - pd.to_numeric(df["转债价格"])
np.std(sorted(df["ridiff"])[10:-10]), np.mean(sorted(df["ridiff"])[10:-10])
# xa估值和市价比较, 实践证明简单的期权估值一样能得到不错的效果，整体估值略微偏低，这是因为没考虑其他几个期权要素的贡献

(7.38055079710973, -3.128113244642439)

In [12]:
df["ridiff"] = pd.to_numeric(df["tvalue"]) - pd.to_numeric(df["内在价值"])
np.std(sorted(df["ridiff"])[10:-10]), np.mean(sorted(df["ridiff"])[10:-10])
# xa 估值和富投估值的比较， 平均水平下估值几乎一致，xa 的估值系统完全可用

(2.4782227297597, 0.08819892306550775)

In [13]:
# 总体表格供参考，英文项目来自 xa 估值，中文项目来自富投
df

,序号,转债名称,转债代码,进展/上市日期,股票名称,股票代码,行业,子行业,转债价格,涨跌,...,双低指数,MA20乖离,热门度,转债名称,bvalue,ovalue,tvalue,votality,rate,ridiff
0,1,春秋转债,113577,2020-05-13,春秋电子,603890,电子,电子行业,119.14,-0.68%,...,131,-1.9%,3,春秋转债,89.282684,30.792936,120.075620,0.22,0.049750,-3.174380
1,2,N聚飞转,123050,2020-05-15,聚飞光电,300303,电子,LED行业,121.7,21.7%,...,128,13.0%,6,N聚飞转,93.782766,37.107186,130.889952,0.22,0.049750,0.149952
2,3,维尔转债,123049,2020-05-12,维尔利,300190,城市公用设施,三废处理,112,-0.53%,...,133,-1.8%,8,维尔转债,94.300367,18.832452,113.132819,0.21,0.049750,-0.167181
3,4,凌钢转债!,110070,2020-05-13,凌钢股份,600231,钢铁,炼钢,100.81,-0.98%,...,124,-1.9%,6,凌钢转债!,95.558688,9.943120,105.501808,0.19,0.036832,1.341808
4,5,华统转债,128106,2020-05-08,华统股份,002840,农业,农产品加工,121.6,-1.22%,...,135,3.5%,5,华统转债,91.829758,29.193664,121.023422,0.19,0.036832,0.113422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,245,广汽转债!,113009,2016-02-04,广汽集团,601238,汽车,乘用车,112,-0.48%,...,155,0.1%,14,广汽转债!,103.854028,0.466917,104.320945,0.17,0.020970,0.790945
245,246,九州转债!,110034,2016-01-29,九州通,600998,医药,医药商业,117.17,-1.26%,...,137,-4.0%,29,九州转债!,105.628312,8.902611,114.530922,0.18,0.022723,4.210922
246,247,国贸转债!,110033,2016-01-19,厦门国贸,600755,交通运输,交通运输行业,110.06,-0.04%,...,136,-0.4%,18,国贸转债!,106.085333,2.909367,108.994700,0.17,0.020970,2.004700
247,248,航信转债!!,110031,2015-06-30,航天信息,600271,计算机及应用,软件开发,111.1,-0.14%,...,144,-6.9%,15,航信转债!!,106.376583,0.650574,107.027157,0.17,0.019019,0.827157
